In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from textblob import TextBlob
import string
from string import punctuation
from nltk.corpus import stopwords
from statistics import mean
from heapq import nlargest
from wordcloud import WordCloud
!pip install rouge
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
import os,glob,re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def list_to_string(strings):
    """
Преобразуем список строк в одну строку.
:param strings: Список строк.
:return: Одиночная строка.
    """
    return "".join(strings)

def text_to_word_list(text):
    return text.split()

def text_to_sentence_list(text):
    import nltk
    nltk.download('punkt')
    from nltk.tokenize import sent_tokenize
    return [sentence.split() for sentence in sent_tokenize(text)]

In [ ]:
import re
rouge1_f1_5 = []
rouge1_f1_10 = []
rouge1_f1_15 = []
rouge1_f1_20 = []

bleu_5 = []
bleu_10 = []
bleu_15 = []
bleu_20 = []

sum_len_w = []
sum_len_l = []

In [ ]:
path = "/content/drive/My Drive/texts/short"
for filename in glob.glob(os.path.join(path, '*.txt')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f: 
      text: str = f.read().replace('\n', '').replace('\r', '')
    
    num_clusters = 5
    while num_clusters < 21:
        keep = string.ascii_letters + string.digits + " " + "."
        text = list_to_string(text).translate(str.maketrans("", "", string.punctuation.replace(".", "")))
        text = text.lower()

        text = sent_tokenize(text)

        max_clusters = len(text)
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(text)
            
        # Определим оптимальное количество кластеров, используя метод elbow
        sum_of_squared_distances = []
        for num_clusters in range(1, max_clusters+1):
            kmeans = KMeans(n_clusters=num_clusters, random_state=0)
            kmeans = kmeans.fit(X)
            sum_of_squared_distances.append(kmeans.inertia_)

        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(text)

        # Выполним кластеризацию K-средних значений
        kmeans = KMeans(n_clusters=num_clusters, random_state=0)
        clusters = kmeans.fit_predict(X)

        # Отнесем каждое предложение к группе
        labels = kmeans.labels_

        # Создаем краткое изложение, выбрав по одному предложению из каждого кластера
        summary = []
        for i in range(num_clusters):
            cluster_sentences = [text[j] for j in range(len(text)) if labels[j] == i]
            summary.append(cluster_sentences[0])

        len(summary)
        paragraph = ' '.join(summary)

        rouge = Rouge()
        scores = rouge.get_scores(paragraph, list_to_string(text))

        result = summary
        summary_text = list_to_string(result)
        predicted_summary = text_to_word_list(summary_text)
        reference_summary = text
        score = sentence_bleu(reference_summary, predicted_summary)

        sum_len_l.append(len(summary))
        sum_len_w.append(len(list_to_string(summary).split()))
        match num_clusters:
          case 5:
            rouge1_f1_5.append(scores[0]['rouge-1']['f'])
            bleu_5.append(score)
          case 10:
            rouge1_f1_10.append(scores[0]['rouge-1']['f'])
            bleu_10.append(score) 
          case 15:
            rouge1_f1_15.append(scores[0]['rouge-1']['f'])
            bleu_15.append(score)
          case 20:
            rouge1_f1_20.append(scores[0]['rouge-1']['f'])
            bleu_20.append(score)            
        num_clusters = num_clusters+5

In [ ]:
import csv

fields1 = [ 'sum_len_l','sum_len_w']
rows1 = [sum_len_l, sum_len_w]

fields2 = ['rouge1_f1_5','bleu5','rouge1_f1_10','bleu10','rouge1_f1_15''bleu_15','rouge1_f1_20','bleu_20']
rows2 = [rouge1_f1_5,bleu_5,rouge1_f1_10,bleu_10,rouge1_f1_15,bleu_15,rouge1_f1_20,bleu_20]

from itertools import zip_longest
export_data = zip_longest(*rows1, fillvalue = '')
with open('len_short_textRank.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields1))
      wr.writerows(export_data)
myfile.close()


export_data = zip_longest(*rows2, fillvalue = '')
with open('metric_short_textRank.csv', 'w', encoding="utf-8", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow((fields2))
      wr.writerows(export_data)
myfile.close()